# ShotGun sequencing
## Extracting sequence

We work with the DNA sequence of SARS-COV2

In [1]:
import numpy as np
path_DNA=".\data\sars_cov_2.txt"
with open(path_DNA) as f:
    full_sequence = f.read()
f.close()
full_sequence=full_sequence.replace('\n','')
print("#Nucleotide:",len(full_sequence))
print(full_sequence[:400])
full_sequence=full_sequence[:1000]

#Nucleotide: 29903
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACAT


## Reads creation
We create a function able to split a sequence into short reads of a length following a normal law $N(read\_length,read\_sigma^2)$

In [2]:
read_length=150
read_sigma=50
fold_coverage=5

def create_reads(read_length=150,read_sigma=50,fold_coverage=5):
	reads_list=[]
	for i in range(fold_coverage):
		n=0
		while n<len(full_sequence):
			rand_int=int(np.random.normal(read_length,read_sigma))
			if n+rand_int<len(full_sequence):
				read=full_sequence[n:n+rand_int]
			else:
				read=full_sequence[n:]
			n+=rand_int
			reads_list.append(read)
	np.random.shuffle(reads_list)
	# reads_list=np.array(reads_list)
	return reads_list

In [3]:
reads=create_reads(read_length,read_sigma,fold_coverage)

## Reads assembly

To simulate shot gun sequencing an reassemble the sequence we create the function *overlap*:
>This function take two reads read_A and read_B, and check if these two sequence overlap. If they do the function return the combined sequence of the two read maximizing the overlap between the two reads. If the two reads do not overlap the function retrun 0.

*combine_with_coverage*:
>This function compare a *read* with all the reads of an array *reads* via *overlap* function. Then the finction return the best combined sequence in term of overlapping.

In [4]:
def overlap(read_A,read_B):
	max_coverage=0
	NA=len(read_A)
	NB=len(read_B)
	if NA<NB:
		short_read=read_A
		N_short=NA
		long_read=read_B
		N_long=NB
	else:
		short_read=read_B
		N_short=NB
		long_read=read_A
		N_long=NA
	cov=0
	i_cov=0
	# print(N_short,N_long)
	for i in range(1,N_short+N_long):
		if i<N_short:
			segment_1=short_read[N_short-i:N_short]
			segment_2=long_read[0:i]
			if segment_1==segment_2:
				if cov<i:
					cov=i
					i_cov=i
		elif i>N_long:
			segment_1=short_read[0:N_short+N_long-i]
			segment_2=long_read[i-N_short:]
			if segment_1==segment_2:
				if cov<N_short+N_long-i:
					cov=N_short+N_long-i
					i_cov=i
		else:
			segment_1=short_read
			segment_2=long_read[i-N_short:i]
			if segment_1==segment_2:
				if cov<N_short:
					cov=N_short
					i_cov=i
					return(long_read,cov)
	# 	print(i,segment_1,segment_2)
	# print(i_cov)
	if i_cov<N_short:
		return(short_read+long_read[i_cov:],cov)
	elif i_cov>N_long:
		return(long_read+short_read[N_short+N_long-i_cov:],cov)
	else:
		return(0,cov)

# read_A='AAAAAAAAAAAAAAAAABABA'
# read_B='BABABB'
read_A='BABAAAAAAAAAAAAAAAAAA'
read_B='CCCBABA'
# read_A='AAAAAAAACCVAAAAAAAAAAA'
# read_B='CCVA'
overlap(read_A,read_B)    
        
        
        

('CCCBABAAAAAAAAAAAAAAAAAA', 4)

In [5]:
def combine_with_coverage(read,reads):   
	N=len(reads)
	max_cov=0
	best_seq=0
	i_best=0
	for i in range(1,N):
		seq,cov=overlap(read,reads[i])
		if cov>max_cov:
			max_cov=cov
			best_seq=seq
			i_best=i
	return(best_seq,i_best)


Then we create an algorithm to combine the sequences:
> At each iteration we take the fist read and combine it with the more revelant other read of *reads* thanks to *combine_with_coverage*. Then in *reads* we remplace the current read with the combinasion and remove the second one in *reads*.
> we perform a circular translation of *reads* at each iteration this allow better algorithmic performances. (Avoid to always compare the biggest read to small read an rather increase the size of small reads and reduce the number of elements in list *reads*) 

In [6]:
reads=create_reads(read_length,read_sigma,fold_coverage)

reads_copy=reads
N=len(reads_copy)
i=0
count_flag=0
while len(reads_copy)>1 and i<=N:
	i+=1
	read=reads_copy[0]
	best_seq,i_best=combine_with_coverage(read,reads_copy)
	# if i_best==0:
	# 	# print("Oups")
	# 	# break
	# 	i-=1
	if i_best!=0: #If a read overlap the current read: we remplace read by the combinaision and remove the cobined one
		count_flag=0
		reads_copy.pop(i_best)
		reads_copy[0]=best_seq
	else: # Else, this iteration didn't redure *reads* size 
		i=i-1
		count_flag+=1
		if count_flag>len(reads_copy):
			break
	if len(read)==0: # If the current object is empty (no overlap) the empty list is deleted, and we redo the iteration
		reads_copy.pop(0)
	else: # We perform a circular translation for better performences
		reads_copy=reads_copy[1:]+[reads_copy[0]]
	if i%100==0: #Counter to indicate algo progress
		print(int(i/N*100),"% ",len(reads_copy),"/",N)

# print(i_best)


**Finnally** we optain a single read in *reads* with is the de novo assembly, comparing this assembly with the reference shows a 100% reconstitution in less then 2 minutes.

In [7]:
len(reads_copy)

1

In [8]:
for i in range(len(reads_copy)):
	print(len(reads_copy[i]),"/",len(full_sequence))

1000 / 1000


In [9]:
len(full_sequence)


1000

In [11]:
full_sequence==reads_copy[0]

True